In [1]:
import time

import pandas as pd
import numpy as np
import joblib
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
trainDataFrame = pd.read_csv('./data/trainLevel1Preds.csv')
#trainLabels = trainDataFrame['TARGET']
#trainFeatures = trainDataFrame.drop(['TARGET'], axis=1)

In [5]:
print trainDataFrame.shape

(76020, 189)


In [10]:
def getRandomSubSet(ratio=.5):
    n_samples = int(trainDataFrame.shape[0]*ratio)
    # shuffle
    trn_shuffled = trainDataFrame.reindex(np.random.permutation(trainDataFrame.index))

    return trn_shuffled[:n_samples]

In [28]:
# cross validated performance
n_folds = 5
n_max_ensemble_models = 50
with_replacement = True
ensemble = []
target = 'TARGET'
models = [x for x in trainDataFrame.columns if x not in [target]]
start = time.time()
for m in range(n_max_ensemble_models):
    models_candidates = [x for x in models if x not in ensemble] if not with_replacement else models
    models_candidates_scores = pd.DataFrame(index=range(n_folds), columns=models_candidates, 
                                            data=np.zeros([n_folds, len(models_candidates)]))
    for i in range(n_folds):
        trn = getRandomSubSet(ratio=.2)
        ensemble_preds = trn[ensemble].mean(axis=1) if ensemble else np.zeros(trn.shape[0])
        ensemble_score = roc_auc_score(trn[target], ensemble_preds)
        for model in models_candidates:
            model_preds = trn[ensemble+[model]].mean(axis=1)
            models_candidates_scores[model].ix[i] = roc_auc_score(trn[target], model_preds) - ensemble_score
            #print "'%s' Contribution Score: %.4f" % (model, (roc_auc_score(trn[target], model_preds) - ensemble_score)) 
    model_score_means = models_candidates_scores.mean()
    model_max_contribution = model_score_means.idxmax()
    if model_score_means[model_max_contribution] <= 0:
        print "Best model '%s' has a contribution of %.4f, thus skipped." % (
            model_max_contribution, model_score_means[model_max_contribution])
        continue
    
    ensemble.append(model_max_contribution)
    print "added '%s' to ensemble w contribution score %.6f" % (model_max_contribution
                                                                , model_score_means[model_max_contribution])

trn = getRandomSubSet(ratio=1.)
ensemble_preds = trn[ensemble].mean(axis=1) if ensemble else np.zeros(trn.shape[0])
ensemble_score = roc_auc_score(trn[target], ensemble_preds)

print "-----------------------"
print "Ensemble: %s" % ensemble
print "Score: %.6f" % ensemble_score
print "Total Time (mins): %.1f" % ((time.time()-start)/60.)

added 'xgb_13' to ensemble w contribution score 0.356300
added 'xgb_11' to ensemble w contribution score 0.001679
added 'xgb_19' to ensemble w contribution score 0.000753
added 'xgb_5' to ensemble w contribution score 0.000406
added 'xgb_18' to ensemble w contribution score 0.000216
added 'xgb_9' to ensemble w contribution score 0.000323
added 'xgb_10' to ensemble w contribution score 0.000174
added 'xgb_6' to ensemble w contribution score 0.000157
added 'rfc_14' to ensemble w contribution score 0.000200
added 'xgb_17' to ensemble w contribution score 0.000080
added 'xgb_11' to ensemble w contribution score 0.000088
added 'xgb_17' to ensemble w contribution score 0.000038
added 'xgb_0' to ensemble w contribution score 0.000069
added 'xgb_14' to ensemble w contribution score 0.000023
added 'xgb_3' to ensemble w contribution score 0.000107
added 'xgb_11' to ensemble w contribution score 0.000078
added 'xgb_10' to ensemble w contribution score 0.000077
added 'rfc_10' to ensemble w contrib

In [22]:
models_candidates_scores

,gbc_0,gbc_1,gbc_2,gbc_3,gbc_4,gbc_5,gbc_6,gbc_7,gbc_8,gbc_9,...,fnn_10,fnn_11,fnn_12,fnn_13,fnn_14,fnn_15,fnn_16,fnn_17,fnn_18,fnn_19
0,-0.000489,-0.000553,-0.000586,-0.000727,-0.000643,-0.000536,-0.000765,-0.000698,-0.000604,-0.000343,...,-0.001393,-0.001261,-0.000936,-0.001168,-0.001065,-0.001098,-0.001302,-0.001248,-0.001218,-0.001203
1,-0.000278,-0.000183,-0.000344,-0.000467,-0.000293,-0.000369,-0.000323,-0.000314,-0.000408,-0.000294,...,-0.000799,-0.000797,-0.000743,-0.000740,-0.000785,-0.000772,-0.000745,-0.000723,-0.000798,-0.000869
2,-0.000621,-0.000438,-0.000540,-0.000374,-0.000513,-0.000217,-0.000331,-0.000409,-0.000490,-0.000228,...,-0.001092,-0.000884,-0.000883,-0.001005,-0.000906,-0.000953,-0.000996,-0.000941,-0.001047,-0.001066
3,-0.000623,-0.000515,-0.000763,-0.000578,-0.000667,-0.000384,-0.000696,-0.000687,-0.000512,-0.000463,...,-0.000960,-0.000917,-0.000861,-0.001044,-0.000915,-0.000963,-0.001157,-0.000917,-0.000983,-0.000969
4,-0.000587,-0.000621,-0.000649,-0.000622,-0.000678,-0.000604,-0.000577,-0.000720,-0.000706,-0.000181,...,-0.001236,-0.001203,-0.000930,-0.001229,-0.001089,-0.001105,-0.001168,-0.001220,-0.001366,-0.001231


In [25]:
print X_train.shape
print y_train.shape
print X_val.shape
print y_val.shape

(38000, 1)
(38000,)
(38020, 1)
(38020,)


In [20]:
confusion_matrix(y_val, clf.predict(X_val))

array([[2596,  404],
       [ 510,  998]])

### train on all data and save feature_importances_

In [30]:
model = GradientBoostingClassifier(max_features=7)
model.fit(trainFeatures, trainLabels)


GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=7, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [32]:
feat_imps_for_level1preds_w_gbc_mf7_all_trainset_sorted = sorted(zip(trainFeatures.columns, model.feature_importances_), key=lambda x: x[1], reverse=True)

In [33]:
feat_imps_for_level1preds_w_gbc_mf7_all_trainset_sorted

[('rfc_5', 0.022271219436144626),
 ('xgb_4', 0.020745370418321536),
 ('rfc_17', 0.017732956220901371),
 ('xgb_9', 0.017092360257458746),
 ('xgb_11', 0.016114674394330394),
 ('svm_16', 0.0160096314446889),
 ('rfc_2', 0.015823938142816171),
 ('xgb_17', 0.015756475145877455),
 ('xgb_12', 0.015720812054844834),
 ('rfc_18', 0.01550412648583859),
 ('rfc_13', 0.015199231880057527),
 ('xgb_8', 0.014816468129873922),
 ('svm_15', 0.0132701715292295),
 ('lr_19', 0.013166665953654186),
 ('fnn_10', 0.012978292938358766),
 ('rfc_3', 0.012093217772584007),
 ('gbc_7', 0.011777018041079928),
 ('gbc_2', 0.011646806421891431),
 ('svm_12', 0.011437337872007118),
 ('xgb_10', 0.011201223101746436),
 ('xgb_14', 0.011070345072233558),
 ('xgb_2', 0.011054223494070066),
 ('xgb_5', 0.010904807731260135),
 ('xgb_6', 0.01078255866133758),
 ('svm_17', 0.01054030719700017),
 ('gbc_18', 0.010350142038380193),
 ('svm_0', 0.010195283386965192),
 ('svm_9', 0.0099980222730563176),
 ('svm_5', 0.0094899618469975345),
 ('et

### calculate auc of each (single) feature

In [59]:
features = trainFeatures.columns
aucs = {}
for clf in features:
    aucs[clf] = roc_auc_score(trainLabels, trainFeatures[clf])
    print "%s: %.4f" % (clf, aucs[clf])

gbc_0: 0.8495
gbc_1: 0.8501
gbc_2: 0.8493
gbc_3: 0.8503
gbc_4: 0.8483
gbc_5: 0.8506
gbc_6: 0.8496
gbc_7: 0.8488
gbc_8: 0.8503
gbc_9: 0.8515
gbc_10: 0.8494
gbc_11: 0.8500
gbc_12: 0.8501
gbc_13: 0.8502
gbc_14: 0.8492
gbc_15: 0.8497
gbc_16: 0.8489
gbc_17: 0.8508
gbc_18: 0.8501
gbc_19: 0.8498
xgb_0: 0.8544
xgb_1: 0.8543
xgb_2: 0.8548
xgb_3: 0.8558
xgb_4: 0.8532
xgb_5: 0.8554
xgb_6: 0.8549
xgb_7: 0.8542
xgb_8: 0.8550
xgb_9: 0.8558
xgb_10: 0.8557
xgb_11: 0.8555
xgb_12: 0.8551
xgb_13: 0.8555
xgb_14: 0.8554
xgb_15: 0.8544
xgb_16: 0.8550
xgb_17: 0.8558
xgb_18: 0.8545
xgb_19: 0.8555
etc_0: 0.8508
etc_1: 0.8498
etc_2: 0.8503
etc_3: 0.8513
etc_4: 0.8502
etc_5: 0.8514
etc_6: 0.8503
etc_7: 0.8503
etc_8: 0.8512
etc_9: 0.8517
etc_10: 0.8509
etc_11: 0.8505
etc_12: 0.8494
etc_13: 0.8509
etc_14: 0.8511
etc_15: 0.8513
etc_16: 0.8515
etc_17: 0.8515
etc_18: 0.8513
etc_19: 0.8502
rfc_0: 0.8528
rfc_1: 0.8521
rfc_2: 0.8527
rfc_3: 0.8530
rfc_4: 0.8520
rfc_5: 0.8533
rfc_6: 0.8527
rfc_7: 0.8527
rfc_8: 0.8534
rfc_

In [62]:
feature_auc_scores_sorted = sorted(aucs.items(), key=lambda x: x[1], reverse=True)

In [63]:
joblib.dump(feature_auc_scores_sorted, './data/feature_auc_scores_sorted.joblib')

['./data/feature_auc_scores_sorted.joblib']

In [92]:
zip(range(1,189), feature_auc_scores_sorted)

[(1, ('xgb_17', 0.85583854994763331)),
 (2, ('xgb_9', 0.85582707331117824)),
 (3, ('xgb_3', 0.8557792065622265)),
 (4, ('xgb_10', 0.85568770309616826)),
 (5, ('xgb_11', 0.85554351319471234)),
 (6, ('xgb_19', 0.85552120193955294)),
 (7, ('xgb_13', 0.85551561501912832)),
 (8, ('xgb_14', 0.85544431689894163)),
 (9, ('xgb_5', 0.85543157899357258)),
 (10, ('xgb_12', 0.85510617389038934)),
 (11, ('xgb_16', 0.85504473142567072)),
 (12, ('xgb_8', 0.85503081876441756)),
 (13, ('xgb_6', 0.85491032660326316)),
 (14, ('xgb_2', 0.85477536172281787)),
 (15, ('xgb_18', 0.8544607821317044)),
 (16, ('xgb_0', 0.85443558179666779)),
 (17, ('xgb_15', 0.85441539238740716)),
 (18, ('xgb_1', 0.85433304791925779)),
 (19, ('xgb_7', 0.85418351925317437)),
 (20, ('rfc_17', 0.85410663648922192)),
 (21, ('rfc_10', 0.85377899343054664)),
 (22, ('rfc_16', 0.85345428498492237)),
 (23, ('rfc_8', 0.85339112364289282)),
 (24, ('rfc_18', 0.85334475721201763)),
 (25, ('rfc_9', 0.85333918167488654)),
 (26, ('rfc_5', 0.8532

## Cross Validate with best auc estimators:

In [ ]:
# helper function to calculate weighted averages
def wavg(row):
  avg = 0.
  for feat, score in feats:
    avg += (row[feat] * score) / float(sum([score for feat, score in feats]))
  return avg

In [85]:
# helper function for minmax() predictions
# minmax(), firstly makes a hard vote on all predictors, decides a class (ie. 0 or 1), and then, 
# sets the predict_proba score with max() if class is 1 and min() if class is zero.
def minmax(row):
    min_pred = 2
    max_pred = -1
    n_preds_1 = 0
    for feat, score in feats:
        if row[feat] > .5:
            n_preds_1 += 1
        if row[feat] < min_pred:
            min_pred = row[feat]
        if row[feat] > max_pred:
            max_pred = row[feat]
    if n_preds_1 > n_preds_total / 2.:
        return max_pred
    else:
        return min_pred    
    # runs too slowly:
    #n_preds_1 = len(row[features][row[features] > 0.5])
    #if n_preds_1 > n_preds_total / 2.:
    #    return row[features].max()
    #else:
    #    return row[features].min()

In [106]:
# cross validated performance
n_folds = 100
weighted = "imp_score" # None or "imp_score"
pred_func = wavg # wavg or minmax
val_scores = []
r_feats = [1,20,40,54,81] # must be list of 1-based indexes of required features

n_feats = [ x-1 for x in r_feats ] # convert to 0-based index values
feats = []
for ix in n_feats:
    if weighted is None:
        feats.append( (feature_auc_scores_sorted[ix][0], 1.) )
    elif weighted == "imp_score":
        feats.append( (feature_auc_scores_sorted[ix][0], feature_auc_scores_sorted[ix][1]) )
features = [ feat for feat, score in feats ]
n_preds_total = len(feats)
start = time.time()
for i in range(n_folds):
    X_val, y_val = getBalancedTrainAndValidationSets(val_only=True)

    preds_val = X_val.apply(pred_func, axis=1)
    val_score = roc_auc_score(y_val, preds_val)

    print "%3d validation auc: %.4f" % (i, val_score)
    val_scores.append(val_score)

print "-----------------------"
print "Best AUC Estimators Voting Classifier with features: %s, weights: %s, n_folds: %d, pred_func: %s" % (
        str(n_feats), str(weighted), n_folds, pred_func.__name__)
print "Mean: %.4f" % np.mean(val_scores)
print "Std: %.4f" % np.std(val_scores)
print "Total Time (mins): %.1f" % ((time.time()-start)/60.)

  0 validation auc: 0.8521
  1 validation auc: 0.8508
  2 validation auc: 0.8549
  3 validation auc: 0.8511
  4 validation auc: 0.8553
  5 validation auc: 0.8539
  6 validation auc: 0.8522
  7 validation auc: 0.8525
  8 validation auc: 0.8515
  9 validation auc: 0.8490
 10 validation auc: 0.8565
 11 validation auc: 0.8537
 12 validation auc: 0.8538
 13 validation auc: 0.8520
 14 validation auc: 0.8504
 15 validation auc: 0.8530
 16 validation auc: 0.8536
 17 validation auc: 0.8548
 18 validation auc: 0.8590
 19 validation auc: 0.8543
 20 validation auc: 0.8519
 21 validation auc: 0.8538
 22 validation auc: 0.8534
 23 validation auc: 0.8578
 24 validation auc: 0.8498
 25 validation auc: 0.8535
 26 validation auc: 0.8514
 27 validation auc: 0.8536
 28 validation auc: 0.8594
 29 validation auc: 0.8548
 30 validation auc: 0.8546
 31 validation auc: 0.8608
 32 validation auc: 0.8512
 33 validation auc: 0.8508
 34 validation auc: 0.8577
 35 validation auc: 0.8538
 36 validation auc: 0.8560
 